<a href="https://colab.research.google.com/github/anthonymiglio/YoloV3_Training_Custom_Model/blob/main/YoloV3_Training_Custom_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to YoloV3 Training Custom Model

# Step 0: Enable GPU on your Google Colab notebook

Yolo Model Training runs around 100 times faster on GPU than on CPU processing. So follow the steps:

On the top Left Bar, Click on **Edit**,
then on the drop menu **Notebook settings**
and under **Hardware Accelarator**
select **GPU**, then hit **Save**.

In [ ]:
# Verify if there is CUDA installed on local machine
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


## Step 1: Check if NVIDIA GPU is enabled


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Aug 15 08:06:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Step 2: Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# https://drive.google.com/drive/my-drive
# !ls /mydrive

## Step 3: Clone the Darknet



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

## Step 4: Compile Darknet using Nvidia GPU
This will take same time and don't worry about the number of messages, it's just been compiled.

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make # make darknet(build)

## Step5: Configure Darknet network for training YOLO V3
This makes a copy ofthe original config file to another one, just to keep safe the original and no need to download again.
Class configuration is set for 2 classes: Custom_class_1 and Custom_class_2. Pleas do use white space within class name.

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# # Set to 1 Class
# !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
# !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# !sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
# !sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
# !sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
# !sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

# Set to 2 Classes
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg # 2000 * (n_of_classes)
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg   # n_of_classes = len(classes)
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg   # n_of_classes
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg   # n_of_classes
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg # 3 * (5 + len(classes))
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg # 3 * (5 + len(classes))
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg # 3 * (5 + len(classes))


In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [ ]:
# !echo "costum_class_1" > data/obj.names
!echo -e 'custom_class_1\ncustom_class_2' > data/obj.names
# !echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

## Step 6: Extract Images

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                # text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])        # 1 Class
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4]) # More than 1 Classes
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
# Create image list
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

## Step 7: Start the training
This is the training step and it will take hours, so do not let your computer sleep or hibernate.

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show -map

# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

## Step 8: To Continue from where it stopped, DO NOT click no Step 7, instead click on STEP8



In [ ]:
# To continue from where it stopped
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Optional to run mAP (Mean Average Precison)

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights